In [1]:
# This line configures matplotlib to show figures embedded in the notebook, 
# instead of opening a new window for each figure. More about that later. 
# If you are using an old version of IPython, try using '%pylab inline' instead.
%matplotlib inline
%load_ext snakeviz

import numpy as np
from scipy.optimize import minimize
from scipy.special import expit
import matplotlib.pyplot as plt

from matplotlib.lines import Line2D

import timeit

import pandas as pd

import plotly.plotly as py
from plotly.graph_objs import *
import plotly.tools as tls

In [7]:
# Here are the initial values


# For test use
array12 = np.asarray(np.split(np.random.rand(1,60)[0],12))

[[ 0.78193249  0.74405463  0.05370159  0.64375254  0.9671223 ]
 [ 0.80076329  0.91518557  0.33013206  0.87704451  0.27513224]
 [ 0.06682722  0.62566131  0.6960552   0.50425531  0.26676161]
 [ 0.84152987  0.6574421   0.66056655  0.33425657  0.51805791]
 [ 0.55130428  0.54877911  0.02722155  0.36000264  0.94506057]
 [ 0.27276162  0.15587826  0.66411734  0.28773056  0.02330387]
 [ 0.39817375  0.87047702  0.57703081  0.02910282  0.6793279 ]
 [ 0.41591379  0.69201614  0.55686696  0.58659307  0.7747083 ]
 [ 0.04192126  0.54676091  0.31462627  0.58499502  0.58181971]
 [ 0.71115902  0.63971152  0.27552422  0.12488898  0.06121642]
 [ 0.84263665  0.55655689  0.85712789  0.29016308  0.31761533]
 [ 0.45020022  0.98342282  0.41708101  0.96441744  0.59613387]]


In [15]:
# Here is the activation function

def act(x):
    return expit(x)

In [17]:
# Density matrix in the forms that I wrote down on my Neutrino Physics notebook
# x is a real array of 12 arrays.

init = np.array([1.0,0.0,0.0,0.0])

def rho(x,ti,initialCondition):
    
    elem = np.ones(4)
    
    for i in np.linspace(0,3,4):
        elem[i] = np.sum(ti*x[i*3]*act(ti*x[i*3+1] + x[i*3+2]) )
    
    return init + elem
    

In [21]:
rho(array12,0,init)

array([ 1.,  0.,  0.,  0.])

In [23]:
# Hamiltonian of the problem, in terms of four real components

hamil = 1.0/2*np.array( [  -np.cos(0.2),np.sin(0.2) , np.sin(0.2),np.cos(0.2) ] )
print hamil

[-0.49003329  0.09933467  0.09933467  0.49003329]


In [127]:
# Cost function for each time step

def rhop(x,ti,initialCondition):
    
    rhoprime = np.zeros(4)
    

    
    for i in np.linspace(0,3,4):
        rhoprime[i] = np.sum(x[i*3] * (act(ti*x[i*3+1] + x[i*3+2]) ) ) +  np.sum( ti*x[i*3]* (act(ti*x[i*3+1] + x[i*3+2]) ) * (1 - (act(ti*x[i*3+1] + x[i*3+2])  ) )* x[i*3+1]  )
        
    
    return rhoprime


## This is the regularization

regularization = 0.0001

def costi(x,ti,initialCondition):
    
    rhoi = rho(x,ti,initialCondition)
    rhopi = rhop(x,ti,initialCondition)
    
    costTemp = np.zeros(4)
    
    costTemp[0] = (rhopi[0] + 2.0*rhoi[2]*hamil[1] )**2
    costTemp[1] = ( rhopi[1] - 2.0*rhoi[2]*hamil[0] )**2
    costTemp[2] = ( rhopi[2] + 2.0*rhoi[1]*hamil[0] + hamil[1] * (rhoi[3] - rhoi[0] ) )**2
    costTemp[3] = ( rhopi[3] -2.0*rhoi[2]*hamil[1] )**2
    
#    return np.sum(costTemp)
    
    return np.sum(costTemp) + regularization*np.sum(x**2)
    

In [128]:
costi(array12,0,init)

9.9883486522479821

In [129]:
def cost(x,t,initialCondition):
    
    costTotal = map(lambda t: costi(x,t,initialCondition),t)
    
    return np.sum(costTotal)

In [130]:
cost(array12,np.array([0,1,2]),init)

109.42309784126915

In [137]:
# with ramdom initial guess

initGuess = np.asarray(np.split(np.random.rand(1,60)[0],12))
endpoint = 4
tlin = np.linspace(0,endpoint,11)

costF = lambda x: cost(x,tlin,init)

startSLSQP = timeit.default_timer()
costvFResultSLSQP = minimize(costF,initGuess,method="SLSQP",tol=1e-10)
stopSLSQP = timeit.default_timer()

print stopSLSQP - startSLSQP

print costvFResultSLSQP

45.7348639965
  status: 0
 success: True
    njev: 88
    nfev: 5471
     fun: 0.010829660025341598
       x: array([ -1.63951793e-02,  -6.06342322e-02,  -6.57520419e-04,
        -1.04846449e-01,  -1.93766208e-01,   5.85545979e-02,
         1.74214962e-01,  -8.34566278e-01,  -5.81189597e-02,
         1.63713413e-02,  -5.99325657e-02,  -1.01831749e-03,
         2.39041029e-06,   3.82642836e-05,   2.63981714e-05,
         4.57385909e-06,   1.30334707e-05,   3.42525461e-05,
         2.10113625e-05,   2.78466351e-05,   2.56350780e-05,
         1.05040400e-05,   3.32641504e-05,   3.56824490e-05,
         7.12071396e-06,   3.58586023e-05,   1.11233420e-05,
         6.58642829e-06,   6.44390525e-06,   1.61638303e-05,
         3.73286316e-05,   1.97946281e-06,   3.22797656e-05,
         2.65048958e-05,   9.38600939e-06,   3.74183144e-06,
         2.11591161e-05,   3.70783501e-05,   3.12159689e-05,
         8.61360998e-06,   1.32196228e-06,   3.28762990e-05,
         2.96389912e-05,   7.7519769

In [146]:
xmid = costvFResultSLSQP.get("x")

startSLSQP = timeit.default_timer()
#costvFResultSLSQP = minimize(costF,xmid,method="SLSQP",tol=1e-30,options={"ftol":1e-30,"maxiter":1000})
costvFResultSLSQP = minimize(costF,xmid,method='Nelder-Mead',tol=1e-15,options={"ftol":1e-15, "maxfev": 1000000,"maxiter":1000000})
stopSLSQP = timeit.default_timer()

print stopSLSQP - startSLSQP

print costvFResultSLSQP

68.3899879456
  status: 0
    nfev: 5878
 success: True
     fun: 0.010829658182270658
       x: array([ -1.63705237e-02,  -5.98684580e-02,  -1.06918811e-03,
        -1.04867460e-01,  -1.93720677e-01,   5.81081685e-02,
         1.74195530e-01,  -8.34543904e-01,  -5.79738425e-02,
         1.63705211e-02,  -5.98684009e-02,  -1.06919095e-03,
         6.97938935e-09,   4.87684951e-08,   4.39967966e-08,
         3.94737695e-08,   1.19148283e-07,   5.51069090e-08,
         1.08920174e-07,   6.47133188e-09,   2.15704279e-08,
         5.35411338e-08,   7.44915748e-08,   6.10864091e-08,
         3.35867266e-08,   5.03452984e-08,   4.50906729e-08,
         1.69856760e-08,   3.43991006e-08,   1.02821701e-07,
         1.27636104e-07,  -2.50059953e-11,   4.97127049e-08,
         8.15862520e-08,   4.65230309e-08,   1.35311522e-09,
         1.75314077e-07,   1.50879050e-07,   4.64130184e-08,
         2.49937983e-08,   7.78578121e-10,   3.46443544e-08,
         6.01802525e-08,   2.58465736e-08,  -2.33

In [ ]:
xresult = costvFResultSLSQP.get("x")
#xresult = np.array([-1.37886409,  2.81454922, -0.3571002 ,  0.02582831, -1.05414931,
#       -1.52308153, -2.24747468,  0.33947049, -0.32310112, -1.43887103,
#        0.81176258,  0.05139705, -1.02669705, -0.97236805, -0.27536667,
#        0.34860447, -1.06962772,  0.89978175,  2.39662887, -1.45165477,
#       -1.54636469, -2.79921374, -1.30335793, -0.62844367, -3.04440811,
#       -2.74566393, -2.16222918, -1.60535643, -0.77298204,  0.13848754,
#       -0.36544212,  1.23901581, -0.80586367, -0.30212561, -1.02818302,
#       -2.82928373, -0.80776632, -2.90056107, -2.42432246, -2.87572658,
#       -0.8645904 , -0.59526987, -1.87029203, -1.60957508, -1.83106839,
#        1.07020356, -0.84892132, -0.97053555, -0.2005098 , -0.72422578,
#       -3.32948549, -4.99349947, -3.46242765, -3.52481528, -3.36820222,
#       -4.1848837 , -1.90748847, -2.09206645, -4.10831718,  2.76094325])

print xresult

[ -1.63705237e-02  -5.98684580e-02  -1.06918811e-03  -1.04867460e-01
  -1.93720677e-01   5.81081685e-02   1.74195530e-01  -8.34543904e-01
  -5.79738425e-02   1.63705211e-02  -5.98684009e-02  -1.06919095e-03
   6.97938935e-09   4.87684951e-08   4.39967966e-08   3.94737695e-08
   1.19148283e-07   5.51069090e-08   1.08920174e-07   6.47133188e-09
   2.15704279e-08   5.35411338e-08   7.44915748e-08   6.10864091e-08
   3.35867266e-08   5.03452984e-08   4.50906729e-08   1.69856760e-08
   3.43991006e-08   1.02821701e-07   1.27636104e-07  -2.50059953e-11
   4.97127049e-08   8.15862520e-08   4.65230309e-08   1.35311522e-09
   1.75314077e-07   1.50879050e-07   4.64130184e-08   2.49937983e-08
   7.78578121e-10   3.46443544e-08   6.01802525e-08   2.58465736e-08
  -2.33532356e-07   3.78940941e-08   1.63822862e-08   1.56759893e-07
   7.46905732e-09   1.01075824e-07   1.74167914e-07   6.11468687e-08
   6.80057120e-08   2.04376585e-08   9.80891658e-09   4.52695736e-08
   1.77436554e-07   4.40240909e-08

In [ ]:
rho(xresult,10,init)

array([  9.41982859e-01,  -1.38941461e-01,   3.90286787e-04,
         5.80171529e-02])

In [ ]:
plttlin=np.linspace(0,endpoint,100)
pltdata11 = np.array([])
for i in plttlin:
    pltdata11 = np.append(pltdata11 ,rho(xresult,i,init)[0] )
    
print pltdata11

In [ ]:
MMA_optmize_Vac_pltdata = np.genfromtxt('./assets/homogen/MMA_optmize_Vac_pltdata.txt', delimiter = ',')

plt.figure(figsize=(16,9.36))
plt.ylabel('MMArho11')
plt.xlabel('Time')
plt.plot(np.linspace(0,15,4501),MMA_optmize_Vac_pltdata,"r-",label="MMAVacrho11")
plt.plot(plttlin,pltdata11,"b4-",label="vac_rho11")
plt.show()
#py.iplot_mpl(plt.gcf(),filename="MMA-rho11-Vac-80-60")